# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('words')
nltk.download('stopwords')
nltk.download('wordnet')
englishwords=set(nltk.corpus.words.words())
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
engine = create_engine('sqlite:///InsertDatabaseName.db')
pd.read_sql_table("InsertTableName", engine).head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("InsertTableName", engine)
X = df["message"]
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)  #remove punctutation
    words = word_tokenize(text)  #tokenize text
    words=[x for x in words if x in englishwords and x not in stopwords.words("english")] #remove stop words and non-english words(just in case)
    wordnet_lemmatizer=WordNetLemmatizer()
    words = [wordnet_lemmatizer.lemmatize(w, pos='v') for w in words]
    words = [wordnet_lemmatizer.lemmatize(w, pos='n') for w in words]
    words = [wordnet_lemmatizer.lemmatize(w, pos='a') for w in words]
    words = [wordnet_lemmatizer.lemmatize(w, pos='v') for w in words]
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [8]:
model = pipeline
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test)

In [10]:
col_names=Y.columns
for i in range(len(col_names)):
        print("Category:", col_names[i],"\n", classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
        print('Accuracy of %s: %.2f \n\n' %(col_names[i], accuracy_score(y_test.iloc[:, i].values, y_pred[:,i])))

Category: related 
              precision    recall  f1-score   support

          0       0.64      0.62      0.63       622
          1       0.82      0.83      0.82      1281
          2       0.48      0.62      0.55        24

avg / total       0.76      0.76      0.76      1927

Accuracy of related: 0.76 


Category: request 
              precision    recall  f1-score   support

          0       0.83      0.88      0.85      1257
          1       0.74      0.65      0.69       670

avg / total       0.79      0.80      0.79      1927

Accuracy of request: 0.80 


Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      1926
          1       0.00      0.00      0.00         1

avg / total       1.00      1.00      1.00      1927

Accuracy of offer: 1.00 


Category: aid_related 
              precision    recall  f1-score   support

          0       0.80      0.89      0.84      1185
          1       0.78      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters ={'clf__estimator__n_estimators': [50, 100],
                  'clf__estimator__min_samples_split': [2, 4, 6],
                  'clf__estimator__criterion': ['entropy', 'gini']
                 }

cv = GridSearchCV(pipeline, param_grid=parameters,verbose=2,n_jobs=-1)

In [12]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total=  56.7s
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.3min remaining:    0.0s


[CV]  clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total=  56.0s
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total=  56.1s
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 1.4min
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 1.4min
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__criterion=entropy, clf__estimator

[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 52.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [50, 100], 'clf__estimator__min_samples_split': [2, 4, 6], 'clf__estimator__criterion': ['entropy', 'gini']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [15]:
cv.best_params_

{'clf__estimator__criterion': 'gini',
 'clf__estimator__min_samples_split': 6,
 'clf__estimator__n_estimators': 100}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
y_pred = cv.predict(X_test)
col_names=Y.columns
for i in range(len(col_names)):
        print("Category:", col_names[i],"\n", classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
        print('Accuracy of %s: %.2f \n\n' %(col_names[i], accuracy_score(y_test.iloc[:, i].values, y_pred[:,i])))

Category: related 
              precision    recall  f1-score   support

          0       0.68      0.61      0.64       622
          1       0.82      0.85      0.84      1281
          2       0.48      0.67      0.56        24

avg / total       0.77      0.77      0.77      1927

Accuracy of related: 0.77 


Category: request 
              precision    recall  f1-score   support

          0       0.86      0.87      0.86      1257
          1       0.75      0.73      0.74       670

avg / total       0.82      0.82      0.82      1927

Accuracy of request: 0.82 


Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      1926
          1       0.00      0.00      0.00         1

avg / total       1.00      1.00      1.00      1927

Accuracy of offer: 1.00 


Category: aid_related 
              precision    recall  f1-score   support

          0       0.83      0.86      0.85      1185
          1       0.76      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
for i in range(len(col_names)):
        print('Accuracy of %s: %.2f' %(col_names[i], accuracy_score(y_test.iloc[:, i].values, y_pred[:,i])))

Accuracy of related: 0.77
Accuracy of request: 0.82
Accuracy of offer: 1.00
Accuracy of aid_related: 0.81
Accuracy of medical_help: 0.95
Accuracy of medical_products: 0.97
Accuracy of search_and_rescue: 0.98
Accuracy of security: 0.98
Accuracy of military: 1.00
Accuracy of child_alone: 1.00
Accuracy of water: 0.98
Accuracy of food: 0.96
Accuracy of shelter: 0.92
Accuracy of clothing: 0.99
Accuracy of money: 0.99
Accuracy of missing_people: 0.99
Accuracy of refugees: 0.99
Accuracy of death: 0.98
Accuracy of other_aid: 0.87
Accuracy of infrastructure_related: 0.97
Accuracy of transport: 0.98
Accuracy of buildings: 0.96
Accuracy of electricity: 0.99
Accuracy of tools: 1.00
Accuracy of hospitals: 1.00
Accuracy of shops: 1.00
Accuracy of aid_centers: 1.00
Accuracy of other_infrastructure: 0.98
Accuracy of weather_related: 0.92
Accuracy of floods: 0.98
Accuracy of storm: 0.98
Accuracy of fire: 1.00
Accuracy of earthquake: 0.97
Accuracy of cold: 0.99
Accuracy of other_weather: 0.98
Accuracy o

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [29]:
from sklearn.tree import DecisionTreeClassifier

In [30]:
pipeline1 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('dct', MultiOutputClassifier(DecisionTreeClassifier())),
])
model1 = pipeline1
model1.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))])

In [31]:
y_pred1 = model1.predict(X_test)
col_names=Y.columns
for i in range(len(col_names)):
        print('Accuracy of %s: %.2f' %(col_names[i], accuracy_score(y_test.iloc[:, i].values, y_pred1[:,i])))

Accuracy of related: 0.74
Accuracy of request: 0.77
Accuracy of offer: 1.00
Accuracy of aid_related: 0.76
Accuracy of medical_help: 0.93
Accuracy of medical_products: 0.96
Accuracy of search_and_rescue: 0.96
Accuracy of security: 0.98
Accuracy of military: 0.99
Accuracy of child_alone: 1.00
Accuracy of water: 0.97
Accuracy of food: 0.94
Accuracy of shelter: 0.89
Accuracy of clothing: 0.99
Accuracy of money: 0.98
Accuracy of missing_people: 0.99
Accuracy of refugees: 0.98
Accuracy of death: 0.97
Accuracy of other_aid: 0.82
Accuracy of infrastructure_related: 0.95
Accuracy of transport: 0.97
Accuracy of buildings: 0.95
Accuracy of electricity: 0.99
Accuracy of tools: 0.99
Accuracy of hospitals: 1.00
Accuracy of shops: 0.99
Accuracy of aid_centers: 0.99
Accuracy of other_infrastructure: 0.97
Accuracy of weather_related: 0.89
Accuracy of floods: 0.97
Accuracy of storm: 0.97
Accuracy of fire: 0.99
Accuracy of earthquake: 0.96
Accuracy of cold: 0.99
Accuracy of other_weather: 0.97
Accuracy o

### 9. Export your model as a pickle file

In [32]:
import pickle

In [33]:
pickle.dump(cv, open("mymodel", "wb"))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.